In [1]:
import os
from getpass import getpass

import openai

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

In [2]:
import pandas as pd
import phoenix as px
from phoenix.trace import TraceDataset, using_project


# Display the complete contents of dataframe cells.
# pd.set_option("display.max_colwidth", None)

In [3]:
!sleep 10

In [4]:
testset_hex = "e4de504721b64dd1b1b2cd762b46b628"

In [5]:
with using_project("ragas-testset"):
    trace_dataset = TraceDataset.load(testset_hex, directory='./data')
    session = px.launch_app(trace=trace_dataset)

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


Unknown project: UHJvamVjdDoy

GraphQL request:4:3
3 | ) {
4 |   node(id: $id) {
  |   ^
5 |     __typename
Traceback (most recent call last):
  File "/home/peter-legion-wsl2/peter-projects/regen-ai/regen-requester/.venv/lib/python3.10/site-packages/graphql/execution/execute.py", line 521, in execute_field
    result = resolve_fn(source, info, **args)
  File "/home/peter-legion-wsl2/peter-projects/regen-ai/regen-requester/.venv/lib/python3.10/site-packages/strawberry/schema/schema_converter.py", line 692, in _resolver
    return _get_result_with_extensions(
  File "/home/peter-legion-wsl2/peter-projects/regen-ai/regen-requester/.venv/lib/python3.10/site-packages/strawberry/schema/schema_converter.py", line 678, in extension_resolver
    return reduce(
  File "/home/peter-legion-wsl2/peter-projects/regen-ai/regen-requester/.venv/lib/python3.10/site-packages/strawberry/schema/schema_converter.py", line 673, in wrapped_get_result
    return _get_result(
  File "/home/peter-legion-wsl2/pet

In [6]:
from llama_index.core import SimpleDirectoryReader

dir_path = "./data/prompt-engineering-papers"
reader = SimpleDirectoryReader(dir_path, num_files_limit=2)
documents = reader.load_data()

In [7]:
len(documents)

41

In [8]:
documents[1]

Document(id_='344adc79-a68f-4ed2-82a1-0e085a4a412f', embedding=None, metadata={'page_label': '2', 'file_name': '1605.08386v1.Heat_bath_random_walks_with_Markov_bases.pdf', 'file_path': '/home/peter-legion-wsl2/peter-projects/regen-ai/nbs/data/prompt-engineering-papers/1605.08386v1.Heat_bath_random_walks_with_Markov_bases.pdf', 'file_type': 'application/pdf', 'file_size': 289178, 'creation_date': '2024-04-13', 'last_modified_date': '2024-04-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='2 CAPRICE STANLEY AND TOBIAS WINDISCH\nreached by a random walk that uses moves from M, whereas for the continuous version, a\nrandom sampling from the unit sphere suﬃces. However, in man y situations where a Markov\nbasis is known, the heat-bath random walk is evid

Set handler for `Llama Index` and `Langchain`(Ragas uses Langchain)

In [9]:

from llama_index.core import set_global_handler
from phoenix.trace.langchain import LangChainInstrumentor


# Setup instrumentation for both llama-index and LangChain (used by Ragas)
set_global_handler("arize_phoenix")
LangChainInstrumentor().instrument()

In [10]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from phoenix.trace import using_project


def build_query_engine(documents):
    vector_index = VectorStoreIndex.from_documents(
        documents,
        embed_model=OpenAIEmbedding(model="text-embedding-3-small")
    )
    query_engine = vector_index.as_query_engine(similarity_top_k=2)
    return query_engine


with using_project("indexing"):
    # By assigning a project name, the instrumentation will send all the embeddings to the indexing project
    query_engine = build_query_engine(documents)

In [11]:
my_traces = px.Client().get_trace_dataset(project_name="indexing").save(directory="./data")

💾 Trace dataset saved to under ID: d1949d6b-5d51-4a53-804f-b48135d4323d
📂 Trace dataset path: data/trace_dataset-d1949d6b-5d51-4a53-804f-b48135d4323d.parquet


In [12]:
my_traces.hex

'd1949d6b5d514a53804fb48135d4323d'

In [13]:
px.Client().query_spans(project_name="indexing").head(n=1)


,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,conversation,context.trace_id,context.span_id,attributes.embedding.embeddings,attributes.embedding.model_name
context.span_id,,,,,,,,,,,,,
89382bb17e9c9a76,embedding,EMBEDDING,None,2024-04-14T11:11:45.367154+00:00,2024-04-14T11:11:46.651350+00:00,OK,,[],None,7a37c265ef8c0ec980a724f88bd5c388,89382bb17e9c9a76,"[{'embedding.vector': [-0.027629341930150986, ...",text-embedding-3-small


In [14]:
px.Client().query_spans(project_name="indexing")["attributes.embedding.embeddings"]

context.span_id
89382bb17e9c9a76    [{'embedding.vector': [-0.027629341930150986, ...
ad213b274c36fec0                                                  NaN
3284038e0f66b416                                                  NaN
4b2c3b373ea223a3                                                  NaN
2d2c8979a965b6a9                                                  NaN
1db8d433624a6e68                                                  NaN
243a953d7261709c                                                  NaN
ec1a7a1ed6048d4b                                                  NaN
66fb1bb06668a191                                                  NaN
4a22614575d4ece9                                                  NaN
6e8b9c1e3ad8f337                                                  NaN
51a5e5545a729f73                                                  NaN
3c99e6b55d5b8eab                                                  NaN
ca53e4115649cc41                                                  NaN
9e0d

Embeddings to dataframe for use in visualization

In [15]:
from phoenix.trace.dsl.helpers import SpanQuery

client = px.Client()
corpus_df = px.Client().query_spans(
    SpanQuery().explode(
        "embedding.embeddings",
        text="embedding.text",
        vector="embedding.vector",
    ),
    project_name="indexing",
)
corpus_df.head()

text  \
context.span_id  position                                                      
89382bb17e9c9a76 0         page_label: 1\nfile_path: /home/peter-legion-w...   
                 1         page_label: 2\nfile_path: /home/peter-legion-w...   
                 2         page_label: 2\nfile_path: /home/peter-legion-w...   
                 3         page_label: 3\nfile_path: /home/peter-legion-w...   
                 4         page_label: 3\nfile_path: /home/peter-legion-w...   

                                                                      vector  
context.span_id  position                                                     
89382bb17e9c9a76 0         [-0.027629341930150986, 0.0020648075733333826,...  
                 1         [-0.008014542050659657, -0.0056237997487187386...  
                 2         [0.022452350705862045, 0.012023705057799816, 0...  
                 3         [-0.004942848347127438, -0.01833670400083065, ...  
                 4         [-0.0169712845236063, 0.0017904225969687104, 0...

In [16]:
test_df = pd.read_csv(f"./data/{testset_hex}-testset.csv")
test_df.head()

,Unnamed: 0,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,0,What aspects of training strategies are discus...,['ment aims to improve the scalability and eff...,The context discusses aspects of training stra...,simple,"[{'page_label': '13', 'file_name': '2301.00234...",True
1,1,How did Li et al. (2023d) enhance long-range l...,"[' (Zhou et al., 2023), and itera-\ntive forwa...",Li et al. (2023d) enhanced long-range language...,simple,"[{'page_label': '12', 'file_name': '2301.00234...",True


In [17]:
import pandas as pd
from datasets import Dataset
from phoenix.trace import using_project
from tqdm.auto import tqdm


def generate_response(query_engine, question):
    response = query_engine.query(question)
    return {
        "answer": response.response,
        "contexts": [c.node.get_content() for c in response.source_nodes],
    }


def generate_ragas_dataset(query_engine, test_df):
    test_questions = test_df["question"].values
    responses = [generate_response(query_engine, q) for q in tqdm(test_questions)]

    dataset_dict = {
        "question": test_questions,
        "answer": [response["answer"] for response in responses],
        "contexts": [response["contexts"] for response in responses],
        "ground_truth": test_df["ground_truth"].values.tolist(),
    }
    ds = Dataset.from_dict(dataset_dict)
    return ds


with using_project("llama-index"):
    ragas_eval_dataset = generate_ragas_dataset(query_engine, test_df)

ragas_evals_df = pd.DataFrame(ragas_eval_dataset)
ragas_evals_df.head()

  0%|          | 0/2 [00:00<?, ?it/s]

,question,answer,contexts,ground_truth
0,What aspects of training strategies are discus...,Supervised in-context training strategies are ...,[In-context LearningTraining Warmup (§4)Superv...,The context discusses aspects of training stra...
1,How did Li et al. (2023d) enhance long-range l...,Li et al. (2023d) enhanced long-range language...,[2022. Lamda: Language mod-\nels for dialog ap...,Li et al. (2023d) enhanced long-range language...


#### Query Embeds => DF for vis.

In [18]:
# dataset containing embeddings for visualization
query_embeddings_df = px.Client().query_spans(
    SpanQuery().explode("embedding.embeddings", text="embedding.text", vector="embedding.vector"),
    project_name="llama-index",
)
query_embeddings_df.head()

,,text,vector
context.span_id,position,,
c92293e067e1ebb8,0,How did Li et al. (2023d) enhance long-range l...,"[0.01767698861658573, -0.02579161524772644, 0...."
379a70f446283ad1,0,What aspects of training strategies are discus...,"[-0.045932866632938385, 0.020280158147215843, ..."


In [19]:

from phoenix.session.evaluation import get_qa_with_reference

# dataset containing span data for evaluation with Ragas
spans_dataframe = get_qa_with_reference(client, project_name="llama-index")
spans_dataframe.head()

,input,output,reference
context.span_id,,,
d1ffe7d73475a941,How did Li et al. (2023d) enhance long-range l...,Li et al. (2023d) enhanced long-range language...,2022. Lamda: Language mod-\nels for dialog app...
a823a0086a15333c,What aspects of training strategies are discus...,Supervised in-context training strategies are ...,In-context LearningTraining Warmup (§4)Supervi...


#### Evaluate LLM Traces; Eval scores => DF

In [20]:

from phoenix.trace import using_project
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    context_precision,
    context_recall,
    faithfulness,
)

# Log the traces to the project "ragas-evals" just to view
# how Ragas works under the hood
with using_project("ragas-evals"):
    evaluation_result = evaluate(
        dataset=ragas_eval_dataset,
        metrics=[faithfulness, answer_correctness, context_recall, context_precision],
    )
eval_scores_df = pd.DataFrame(evaluation_result.scores)

Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

#### Submit Evals to Phoenix to make visible on spans

In [21]:
from phoenix.trace import SpanEvaluations

# Assign span ids to your ragas evaluation scores (needed so Phoenix knows where to attach the spans).
eval_data_df = pd.DataFrame(evaluation_result.dataset)
assert eval_data_df.question.to_list() == list(
    reversed(spans_dataframe.input.to_list())  # The spans are in reverse order.
), "Phoenix spans are in an unexpected order. Re-start the notebook and try again."
eval_scores_df.index = pd.Index(
    list(reversed(spans_dataframe.index.to_list())), name=spans_dataframe.index.name
)

# Log the evaluations to Phoenix under the project "llama-index"
# This will allow you to visualize the scores alongside the spans in the UI
for eval_name in eval_scores_df.columns:
    evals_df = eval_scores_df[[eval_name]].rename(columns={eval_name: "score"})
    evals = SpanEvaluations(eval_name, evals_df)
    px.Client().log_evaluations(evals, project_name="llama-index")